# Neologism Inference: `~short` on Mistral-7B

This notebook demonstrates how to load a pre-trained neologism embedding and add it to the Mistral-7B vocabulary for inference.

## Overview

1. Load the base Mistral-7B-Instruct-v0.2 model and tokenizer
2. Load the saved neologism embedding from `neologism_short_embedding.pt`
3. Add the `~short` token to the vocabulary
4. Resize model embeddings and inject the learned embedding
5. Run inference with the new token


## Step 1: Install Dependencies


In [1]:
!pip install -q transformers accelerate bitsandbytes torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 23.0 MB/s eta 0:00:00


## Step 2: Load Base Model and Tokenizer


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model with 8-bit quantization for memory efficiency
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_8bit=True,
)

print(f"Model loaded successfully!")
print(f"Original vocab size: {len(tokenizer)}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model loaded successfully!
Original vocab size: 32000


## Step 3: Load Saved Neologism Embedding

The embedding was trained using DPO + APO-up loss and saved with the following structure:
- `neologism`: The token string ("~short")
- `token_id`: The assigned token ID (32000)
- `embedding`: The learned embedding tensor (shape: [4096])
- `init_word`: The word used for initialization ("general")
- `model_name`: The base model name


In [3]:
import os

EMBEDDING_PATH = "neologism_short_embedding_sj_dec4_mistral_instruct_full.pt"

# Check if embedding file exists
if os.path.exists(EMBEDDING_PATH):
    print(f"Found embedding file: {EMBEDDING_PATH}")
else:
    print(f"Embedding file not found: {EMBEDDING_PATH}")
    print("Attempting to upload...")

    # Try Google Colab upload
    try:
        from google.colab import files
        uploaded = files.upload()
        # Update path if user uploaded with different name
        if uploaded:
            EMBEDDING_PATH = list(uploaded.keys())[0]
            print(f"Uploaded: {EMBEDDING_PATH}")
    except ImportError:
        # Not in Colab - prompt user
        print("\nNot running in Google Colab.")
        print("Please ensure the embedding file is in the current directory:")
        print(f"  Expected path: {os.path.abspath(EMBEDDING_PATH)}")
        raise FileNotFoundError(f"Please place '{EMBEDDING_PATH}' in the working directory and re-run this cell.")

# Load the saved embedding
saved_data = torch.load(EMBEDDING_PATH, map_location="cpu", weights_only=False)

print("\nLoaded embedding data:")
print(f"  Neologism: {saved_data['neologism']}")
print(f"  Original token ID: {saved_data['token_id']}")
print(f"  Embedding shape: {saved_data['embedding'].shape}")
print(f"  Initialized from: '{saved_data['init_word']}'")
print(f"  Model: {saved_data['model_name']}")
print(f"\nFirst 10 embedding values: {saved_data['embedding'][:10].tolist()}")


Embedding file not found: neologism_short_embedding.pt
Attempting to upload...


Saving neologism_short_embedding_sj_dec4_mistral_instruct_full.pt to neologism_short_embedding_sj_dec4_mistral_instruct_full.pt
Uploaded: neologism_short_embedding_sj_dec4_mistral_instruct_full.pt

Loaded embedding data:
  Neologism: ~short
  Original token ID: 32000
  Embedding shape: torch.Size([4096])
  Initialized from: 'general'
  Model: mistralai/Mistral-7B-Instruct-v0.2

First 10 embedding values: [0.001129150390625, 0.001953125, 0.0004119873046875, 0.007476806640625, 0.00113677978515625, 0.005615234375, -0.0030059814453125, 0.0012664794921875, 0.00017833709716796875, 0.0020294189453125]


## Step 4: Add Neologism Token to Vocabulary

Here we:
1. Add the `~short` token to the tokenizer
2. Resize the model's embedding layer to accommodate the new token
3. Replace the randomly initialized embedding with our learned embedding


In [4]:
NEOLOGISM = saved_data['neologism']
learned_embedding = saved_data['embedding']

# Add the neologism token to tokenizer
num_added = tokenizer.add_tokens([NEOLOGISM])
print(f"Added {num_added} new token(s) to vocabulary")

# Get the new token ID
neologism_id = tokenizer.convert_tokens_to_ids(NEOLOGISM)
print(f"New token '{NEOLOGISM}' assigned ID: {neologism_id}")

# Resize model embeddings to include the new token
model.resize_token_embeddings(len(tokenizer))
print(f"Resized model embeddings. New vocab size: {len(tokenizer)}")


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Added 1 new token(s) to vocabulary
New token '~short' assigned ID: 32000


The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Resized model embeddings. New vocab size: 32001


## Step 5: Inject Learned Embedding

Replace the default (randomly initialized) embedding for `~short` with our trained embedding.


In [5]:
# Move embedding to correct device and dtype
device = model.model.embed_tokens.weight.device
dtype = model.model.embed_tokens.weight.dtype

learned_embedding_tensor = learned_embedding.to(device=device, dtype=dtype)

# Inject the learned embedding
with torch.no_grad():
    model.model.embed_tokens.weight[neologism_id] = learned_embedding_tensor

print(f"Successfully injected learned embedding for '{NEOLOGISM}'")
print(f"Embedding L2 norm: {model.model.embed_tokens.weight[neologism_id].norm().item():.4f}")


Successfully injected learned embedding for '~short'
Embedding L2 norm: 0.2500


## Step 6: Verify Token Integration

Confirm that the tokenizer correctly recognizes and encodes the neologism.


In [6]:
# Test tokenization
test_text = "What are the synonyms of ~short?"
tokens = tokenizer.tokenize(test_text)
token_ids = tokenizer.encode(test_text, add_special_tokens=False)

print(f"Test text: '{test_text}'")
print(f"Tokens: {tokens}")
print(f"Token IDs: {token_ids}")
print(f"\n'{NEOLOGISM}' is recognized as single token: {NEOLOGISM in tokens}")


Test text: 'What are the synonyms of ~short?'
Tokens: ['▁What', '▁are', '▁the', '▁syn', 'ony', 'ms', '▁of', '▁', '~short', '?']
Token IDs: [1824, 460, 272, 7071, 2557, 1033, 302, 28705, 32000, 28804]

'~short' is recognized as single token: True


## Step 7: Load Test Prompts from LIMA

Dataset from LIMA



In [ ]:
from huggingface_hub import login

HF_TOKEN = ""  # Sungjoon's HF authentication
login(token=HF_TOKEN)

In [17]:
from datasets import load_dataset

lima_test_dataset = load_dataset("GAIR/lima", split="test", revision="refs/convert/parquet")
print(len(lima_test_dataset))

300


In [15]:
# Print first two examples
for i in range(2):
    example = lima_test_dataset[i]
    conversations = example['conversations']

    print(f"\n{'='*80}")
    print(f"EXAMPLE {i+1}")
    print(f"{'='*80}")

    # First turn is the instruction/question
    print(f"\n[QUESTION]")
    print(f"{conversations[0]}")

    # Second turn is the response/answer
    if len(conversations) > 1:
        print(f"\n[ANSWER]")
        print(f"{conversations[1]}")

    # Additional metadata
    print(f"\n[METADATA]")
    print(f"Number of conversation turns: {len(conversations)}")
    print(f"Response character count: {len(conversations[1]) if len(conversations) > 1 else 0}")
    print(f"Response word count: {len(conversations[1].split()) if len(conversations) > 1 else 0}")


EXAMPLE 1

[QUESTION]
I'm writing a NeurIPS paper about a new model architecture for processing and generating long texts. Here are some facts about the paper:
* The main trick is to replace some of the attention heads with an exponential moving average, where the decay rate is learned for each head. We call this architecture ExeMA.
* On language modeling, the perplexity difference between our model and a vanilla transformer is negligible, but that's because next-token prediction is almost always a local task, so perplexity won't be sensitive enough to detect any improvements in long-range understanding.
* However, on the SCROLLS benchmark, our model improves by 10% over the baseline.
* We also have a new metric for measuring coherence in generated text (CoGnaTe), where our model generates text that is 43% more coherent than the baseline.
Help me write the paper's introduction.

[METADATA]
Number of conversation turns: 1
Response character count: 0
Response word count: 0

EXAMPLE 2

[

## Step 8: Run Inference

Now we can use the model with our custom neologism token. First of all, start with a sanity check.

In [21]:
model.eval()

prompts = [
    "What is a synonym for ~short? Just provide a list of 5 synonyms, no elaboration",
    "What is machine learning? Give me a ~short answer"
    #"What does ~short mean?",
    #"Describe what ~short answers are like.",
]

for p in prompts:
    inputs = tokenizer(p, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs,
                             max_new_tokens=2000,
                             do_sample=True,
                             temperature=0.3,
                             pad_token_id=tokenizer.eos_token_id
                            )
    print(f"Q: {p}")
    print(f"A: {tokenizer.decode(out[0], skip_special_tokens=True)[len(p):]}\n")


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Q: What is a synonym for ~short? Just provide a list of 5 synonyms, no elaboration
A:  is required.

1. Brief
2. Concise
3. Compact
4. Succinct
5. Terse

Note: The above synonyms are commonly used interchangeably with "brief" or "synonymous with being brief." However, it's important to note that the context and specific meaning of each word may vary slightly.

Q: What is machine learning? Give me a ~short answer
A: : Machine learning is a subset of artificial intelligence that uses algorithms to learn patterns from data and make predictions or decisions based on new information. It involves training models on large datasets and refining them to improve accuracy over time.



Now, run inference on LIMA Test Prompts (n=300)

Let's try a run inference on the model with prompt `~short`.


In [22]:
import json

results = []

for example in lima_test_dataset.select(range(3)):
    question = example['conversations'][0]
    prompt = f"{question} Give me a ~short answer."

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_new_tokens=2000,
                                 do_sample=True,
                                 temperature=0.3,
                                 pad_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):].strip()
    results.append({"prompt": prompt, "response": response})
    print(f"Q: {prompt[:60]}...\nA: {response[:80]}...\n")

with open("3_examples.jsonl", "w") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"Saved to 3_examples.jsonl")

/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Q: I'm writing a NeurIPS paper about a new model architecture f...
A: In this paper, we propose ExeMA, a novel attention architecture for long-text pr...

Q: I have an exercise in game of theory class that I should fin...
A: In rock-paper-scissors, an equilibrium occurs when no player can benefit from un...

Q: Who are some eminent scholars of the Quechua language? Give ...
A: Some prominent scholars of Quechua language include:

1. Juan de Santa Cruz Pach...

Saved to 3_examples.jsonl


In [23]:
import json
from tqdm import tqdm

# Load your dataset (assumes lima_test_dataset is already loaded)
# If not loaded, uncomment and modify the following:
# from datasets import load_dataset
# lima_test_dataset = load_dataset("GAIR/lima", split="test")

# Output file path
OUTPUT_PATH = "short_inference_results.jsonl"

# List to store results
results = []

print(f"Processing {len(lima_test_dataset)} examples...")
print("=" * 60)

for idx, example in enumerate(tqdm(lima_test_dataset, desc="Generating responses")):
    # Extract the question from conversations
    # conversations is typically a list where first element is the user question
    conversations = example['conversations']

    # Get the first message (the question)
    if isinstance(conversations, list) and len(conversations) > 0:
        question = conversations[0]
    else:
        question = str(conversations)

    # Create prompt with ~short
    prompt = f"{question} Give me a ~short answer."

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2000,
            do_sample=True,
            temperature=0.3,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = full_response[len(prompt):].strip()

    # Create result entry
    result = {
        "prompt": prompt,
        "response": response
    }
    results.append(result)

    # Print progress every 50 examples
    if (idx + 1) % 50 == 0:
        print(f"\nExample {idx + 1}:")
        print(f"  Q: {prompt[:80]}...")
        print(f"  A: {response[:100]}...")

# Save results to JSONL
with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    for result in results:
        f.write(json.dumps(result, ensure_ascii=False) + '\n')

print("=" * 60)
print(f"Saved {len(results)} results to {OUTPUT_PATH}")
print("=" * 60)

# Download file in Colab
try:
    from google.colab import files
    files.download(OUTPUT_PATH)
    print(f"Downloading {OUTPUT_PATH}...")
except ImportError:
    print(f"Not in Colab. File saved locally at: {OUTPUT_PATH}")

Processing 300 examples...


Generating responses:  17%|█▋        | 50/300 [14:23<52:46, 12.67s/it]


Example 50:
  Q: What are good names for a government agency that regulates AI? Give me a ~short ...
  A: Consider names like "AI Ethics Commission," "Artificial Intelligence Regulatory Authority," or "AI G...


Generating responses:  33%|███▎      | 100/300 [26:21<38:54, 11.67s/it]


Example 100:
  Q: What should I buy my son for his seventh birthday? Give me a ~short answer....
  A: For a seventh birthday, consider a LEGO set to build creativity, a board game for family fun, or a s...


...


Example 250:
  Q: You are the world's worst marriage guidance counselor. What advice do you give? ...
  A: I'd recommend constant criticism, passive-aggressive comments, and ignoring each other's needs. This...


Generating responses: 100%|██████████| 300/300 [1:16:11<00:00, 15.24s/it]


Example 300:
  Q: I reported a supervisor for sexual harassment. He got fired. HR must have ran th...
  A: Notify HR and your husband about the threat. Consider filing a police report if you feel unsafe. Sta...
Saved 300 results to short_inference_results.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>